In [1]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [2]:
download_file_from_google_drive('181Ua_0HhqaWlyCeHD6_hFsPZhGbFKbU8', '/content/prepared.zip')

In [ ]:
!unzip prepared.zip

In [4]:
import tensorflow as tf
import sklearn as skl
import librosa as lr
import numpy as np
import logging
import os

In [5]:
logging.warnings.filterwarnings('ignore')

### Genre Selection:

In [6]:
genres = {
    'blues': 0,
    'classical': 1,
    'country': 2,
    'disco': 3,
    'hiphop': 4,
    'jazz': 5,
    'metal': 6,
    'pop': 7,
    'reggae': 8,
    'rock': 9
}

### Hyper Parameters:

In [8]:
train_x = np.load('./prepared/train_x.npy', allow_pickle=False)
val_x = np.load('./prepared/val_x.npy', allow_pickle=False)
test_x = np.load('./prepared/test_x.npy', allow_pickle=False)

### Generate Ground Truth Labels:

In [9]:
train_y = np.zeros(shape=(5 * 700, 10), dtype=float)
val_y   = np.zeros(shape=(5 * 200, 10), dtype=float)
test_y  = np.zeros(shape=(5 * 100, 10), dtype=float)

train_counter = 0
val_counter   = 0
test_counter  = 0

for i in range(5 * 700):
    train_y[train_counter, :] = tf.keras.utils.to_categorical(train_counter // (5*70), num_classes=10)
    train_counter += 1

for i in range(5 * 200):
    val_y[val_counter, :] = tf.keras.utils.to_categorical(val_counter // (5*20), num_classes=10)
    val_counter += 1
        
for i in range(5 * 100):
    test_y[test_counter, :] = tf.keras.utils.to_categorical(test_counter // (5*10), num_classes=10)
    test_counter += 1

In [10]:
train_x = np.moveaxis(train_x, 1, 2)
val_x = np.moveaxis(val_x, 1, 2)
test_x = np.moveaxis(test_x, 1, 2)

In [51]:
class GenreClassifierModel(tf.keras.Model):
    
    def __init__(self):
        super().__init__()
        
        self.L1 = tf.keras.layers.LSTM(128, return_sequences=True)
        self.L2 = tf.keras.layers.LSTM(64, return_sequences=False)
        self.L3 = tf.keras.layers.Dense(units=10, activation="softmax")
        
    def call(self, inputs):
        self.x = self.L1(inputs)
        self.x = self.L2(self.x)
        self.x = self.L3(self.x)
        return self.x

In [52]:
model = GenreClassifierModel()

In [53]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [54]:
hist = model.fit(train_x, train_y, epochs=200)

Epoch 1/200
110/110 [==============================] - 3s 24ms/step - loss: 1.8056 - accuracy: 0.3474
Epoch 2/200
110/110 [==============================] - 3s 23ms/step - loss: 1.5212 - accuracy: 0.4477
Epoch 3/200
110/110 [==============================] - 3s 23ms/step - loss: 1.3815 - accuracy: 0.5086
Epoch 4/200
110/110 [==============================] - 3s 23ms/step - loss: 1.3266 - accuracy: 0.5200
Epoch 5/200
110/110 [==============================] - 3s 23ms/step - loss: 1.2656 - accuracy: 0.5497
Epoch 6/200
110/110 [==============================] - 3s 23ms/step - loss: 1.2428 - accuracy: 0.5631
Epoch 7/200
110/110 [==============================] - 3s 23ms/step - loss: 1.2109 - accuracy: 0.5731
Epoch 8/200
110/110 [==============================] - 3s 23ms/step - loss: 1.1616 - accuracy: 0.6037
Epoch 9/200
110/110 [==============================] - 3s 23ms/step - loss: 1.1350 - accuracy: 0.6040
Epoch 10/200
110/110 [==============================] - 3s 23ms/step - loss: 1.097

In [55]:
model.evaluate(val_x, val_y)

32/32 [==============================] - 0s 11ms/step - loss: 3.3157 - accuracy: 0.4960


[3.315716505050659, 0.4959999918937683]

In [63]:
for i in range(10): # Genres
  for j in range(10): # Songs
    prob = [0 for i in range(10)]
    for k in range(5): # Segment
      example = test_x[50 * i + 5 * j + k, :]
      example = example.reshape(1, m, 40)
      prob[np.argmax(model.predict(example))] += (1.0 / 5.0)
    print(np.argmax(prob))

0
9
0
2
0
9
2
0
6
0
1
1
1
1
1
1
1
1
5
5
0
2
6
2
0
0
0
0
2
0
3
3
3
3
3
3
3
3
3
3
3
3
8
6
8
6
8
3
4
4
9
5
9
9
5
5
5
5
5
5
6
0
9
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
4
3
8
8
9
9
6
2
8
6
0
6
6
9
6
9
6
6
6
2
